In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 데이터 임포팅 
df = pd.read_csv('menu_concated_v1.4')
df.dropna(inplace=True)
menu_list = df['menu'].tolist()

2023-01-17

- [해결] 빈도수 비율 없이, 그냥 맨 뒷 토큰을 사용해보는 실험 해보기

- [해결] soynlp를 통한 토큰화 시도


2023-01-24

- [해결] 커스텀 사전 제작 

- [해결] 맨 뒷 단어 중심 알고리즘 예외처리

- [해결] 초성 기준의 표준화 작업

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 10.9 MB/s eta 0:00:00


In [ ]:
corpus = pd.read_csv('/content/custom_dict_v1.3.txt')

In [ ]:
corpus = corpus['title']

In [ ]:
corpus

0           대라피
1           아이스
2           육개장
3       쉐이주로우피엔
4            참돔
         ...   
2465       마가리타
2466     케이준후라이
2467         장미
2468        뻥튀기
2469        겉절이
Name: title, Length: 2470, dtype: object

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer, RegexTokenizer

tokenizer = RegexTokenizer() ## Regex 기준 토크나이저, 띄어쓰기가 있거나 이어지는 단어가 한국어가 아닐 경우 토큰화

corpus_list = []

for word in corpus:
  corpus_list.append(tokenizer.tokenize(word))

/usr/local/lib/python3.8/dist-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [ ]:
corpus_list

In [ ]:
corpus_dict = {}

for tokens in corpus_list:
  for token in tokens:
    corpus_dict[token] = 1.0

    ## MaxScoreTokenizer에 사용할 커스텀 사전, 점수를 넣을 경우 해당 단어를 토큰으로 분리하는 가중치를 지정
    ## 서울대학교 라는 단어가 있는데, 서울 :1 대학교 : 1 서울대학교 : 1 이처럼 점수가 같을 경우
    ## 단어가 긴 서울대학교를 우선으로 지정해준다

In [ ]:
len(corpus_dict)

5901

In [ ]:
tokenizer = MaxScoreTokenizer(scores=corpus_dict) ## 위에서 만든 사전을 점수 계산으로 지정

In [ ]:
from tqdm import tqdm

menu_list = [tokenizer.tokenize(word) for word in tqdm(menu_list)]

100%|██████████| 51437/51437 [00:02<00:00, 21753.62it/s]


In [ ]:
## 커스텀 사전의 토큰들만 리스트로 추출
corpus_dict_list = corpus_dict.keys()

In [ ]:
from collections import Counter

df['tokens'] = menu_list
# Counter 객체는 리스트요소의 값과 요소의 갯수를 카운트 하여 저장하고 있습니다.
# 카운터 객체는 .update 메소드로 계속 업데이트 가능합니다.
word_counts = Counter()

# 토큰화된 각 리뷰 리스트를 카운터 객체에 업데이트 합니다. 
for tokens in df['tokens']:
  for token in tokens:
    if token in corpus_dict_list: ## 사전에 있는 경우만 빈도수를 체크
      word_counts.update([token])

# 가장 많이 존재하는 단어 순으로 10개를 나열합니다
word_counts.most_common(10)

[('치즈', 1685),
 ('치킨', 1452),
 ('추가', 1426),
 ('라떼', 980),
 ('구이', 894),
 ('새우', 869),
 ('볶음', 808),
 ('정식', 786),
 ('피자', 750),
 ('불고기', 692)]

In [ ]:
## 가장 적게 존재하는 단어 순으로 정렬합니다. 10개만 나타내는 경우
word_counts.most_common()[:-10:-1] 

[('좌종당계', 1),
 ('막고량주', 1),
 ('스크류키스', 1),
 ('메로나주', 1),
 ('샤크주', 1),
 ('썸주', 1),
 ('튀김사리', 1),
 ('유삼슬밥', 1),
 ('깐소육', 1)]

In [ ]:
len(word_counts)

2373

In [ ]:
word_counts_sorted = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

In [ ]:
token_list = word_counts.keys()

In [ ]:
len_1_tokens = []

for token in token_list:
  if len(token) <= 1:
    len_1_tokens.append(token)

In [ ]:
len(token_list)

2373

In [ ]:
len(len_1_tokens)

60

# 사전에 들어있는 단어(음식 이름)만을 이용해서 카테고리로 분류

In [ ]:
for wv in word_counts_sorted:
  if '함바그' in wv[0]:
    print(wv)

('함바그', 5)


In [ ]:
for wv in word_counts_sorted:
  if '함박' in wv[0]:
    print(wv)

('함박', 101)


In [ ]:
for wv in word_counts_sorted:
  if '참이슬' in wv[0]:
    print(wv)

('참이슬', 11)


In [ ]:
from soynlp.hangle import jamo_levenshtein

jamo_levenshtein('함바그', '함박')

1.3333333333333333

- 기존 자모단위 levenshtein 라이브러리에는 한계점이 있다.
- 한국어 특유의 연음 관련한 부분의 비용을 제대로 계산하지 못하는 것이다.

- 이는 자모 단위로 나눌 때, 반드시 초/중/종성의 형식을 맞추기 위해서 종성이 없는 경우에도 특수문자를 넣어주기 때문이라고 생각한다

- 해결을 위해서 자모 단위로 바꾸는 함수부터 수정한다.

In [ ]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    char = chr(
        kor_begin +
        chosung_base * chosung_list.index(chosung) +
        jungsung_base * jungsung_list.index(jungsung) +
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    if not character_is_korean(c):
        return None
    i = ord(c)
    if (jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if (moum_begin <= i <= moum_end):
        return (' ', c, ' ')

    # decomposition rule
    i -= kor_begin
    cho  = i // chosung_base
    jung = ( i - cho * chosung_base ) // jungsung_base 
    jong = ( i - cho * chosung_base - jung * jungsung_base )

    if jongsung_list[jong] == ' ': ## 종성이 없을 경우, 문자를 넣지 않는다
      return (chosung_list[cho], jungsung_list[jung])

    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
            (jaum_begin <= i <= jaum_end) or
            (moum_begin <= i <= moum_end))

In [ ]:
def levenshtein(s1, s2, cost=None, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug=debug)

    if len(s2) == 0:
        return len(s1)

    if cost is None:
        cost = {}

    # changed
    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return cost.get((c1, c2), 1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            # Changed
            substitutions = previous_row[j] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

def custom_jamo_levenshtein(s1, s2, debug=False):
    s1_ = ''.join([comp for c in s1 for comp in decompose(c)])
    s2_ = ''.join([comp for c in s2 for comp in decompose(c)])

    return levenshtein(s1_, s2_)/3 

In [ ]:
custom_jamo_levenshtein('함바그', '함박')

0.3333333333333333

커스텀한 jamo_levenshtein은 함바그와 함박의 비용을 0.3333으로 계산한다. 즉, 연음을 반영한 것이다. 

In [ ]:
## 빈도수 기준 내림차순 정렬한 토큰 사전 리스트
word_counts_list = [x[0] for x in word_counts_sorted]

In [ ]:
## 표준화 결과를 저장할 딕셔너리
std_dict = {}

def jamo_std(word, count_threshold=1, score_threshold=0.5):
  ## count_threshold : 점수 계산 기준 빈도수 기본값 : 1 / 빈도수가 1보다 큰 단어만 점수 계산
  ## score_threshold : 점수 계산 시 범위 기준값 기본값 : 0.5 / 최소점수가 0일 경우 0 + 0.5 의 범위까지 탐색

  # 표준화 결과가 있는 단어일 경우 바로 변환
  if std_dict.get(word) is not None:
    return std_dict[word]

  scores = [(custom_jamo_levenshtein(word, x[0]),x[0], x[1]) for x in word_counts_sorted if x[1] > count_threshold]

  scores_sorted = sorted(scores, key=lambda x: x[0]) ## levenshtein 비용 기준으로 오름차순으로 정렬
  min_score = scores_sorted[0][0] ## 가장 작은 점수(단어 중 비용이 가장 작은 값)

  scores = [x for x in scores_sorted if x[0] <= min_score + score_threshold] ## 최소 비용 + 범위 임계값

  scores_sorted = sorted(scores, key=lambda x: x[2], reverse=True) ## 점수가 같을 경우 빈도수 높은 것을 선택하기 위해 정렬


  std_word = scores_sorted[0][1]
  std_dict[word] = std_word
   
  return std_word

In [ ]:
jamo_std('취김')

'튀김'

In [ ]:
jamo_std('해무람자탕')

'해물탕'

In [ ]:
jamo_std('침치회')

'참치회'

In [ ]:
jamo_std('더치아메이카노')

'아메리카노'

In [ ]:
jamo_std('닭도티탕')

'닭도리탕'

In [ ]:
jamo_std('함바그')

'함박'

In [ ]:
jamo_std('함바그', score_threshold=0.1)

'함바그'

In [ ]:
jamo_std('차이슬')

'아이스'

In [ ]:
jamo_std('차이슬', score_threshold=0.1)

'참이슬'

In [ ]:
custom_jamo_levenshtein('차이슬', '아이스')

0.6666666666666666

In [ ]:
custom_jamo_levenshtein('차이슬', '참이슬')

0.3333333333333333

빈도수 기반이라 아이스 / 참이슬 같은 경우에서 참이슬이라는 고유명사가 제대로 반영되지 않는 부분이 있다.

임계점을 줄이면 해결되지만 그럴 경우 함바그 - 함박 같은 사례를 해결하지 못한다.

- [미해결] 차이슬 -> 참이슬의 문제

In [ ]:
from tqdm import tqdm

menu_cluster = {} ## 메뉴 분류 dict
OOV_list = [] ## OOV 사전
threshold = 0.7 ## 원본 메뉴명에 대한 분류 실패 기준점 / 70%보다 많이 변경되면 분류 실패

for i, tokens in tqdm(enumerate(df['tokens'])):
  menu_c = ''
  menu_origin = df['menu'][i]
  for token in tokens:
    if token in corpus_dict_list: ## 사전에 있는 경우
      ## 토큰 마다 띄어쓰기
      token = jamo_std(token) ## 자모 단위 표준화
      menu_c += token + " " 
    else:
      token_std = jamo_std(token)
      OOV_list.append((token, token_std)) ## 토큰이 사전에 없을 경우 OOV 리스트에 삽입, (원본, 표준화 이후) 형태로 삽입
      
  ## 내부의 어떤 토큰도 사전에 없을 경우 고유명사라고 판단, 모든 토큰을 하나의 메뉴명으로 봄
  ## 사전에 없을 경우 분리가 안되므로, tokens에는 하나의 토큰만 있게 됨
  if menu_c == '':
    menu_c = jamo_std(tokens[-1])
  if len(menu_c) / len(menu_origin) < (1 - threshold): ## 분류된 단어가 전체 단어를 너무 많이 빼버렸을 경우 분류가 실패했다고 판단
    menu_c = menu_origin
  menu_cluster[menu_origin] = menu_c.rstrip() ## 토큰이 사전에 있는 단어(음식 이름)일 경우만 넣는다.
  ## 맨 뒤의 공백은 제거하기 위해 rstrip()을 사용

51437it [33:33, 25.54it/s]


33분 33초가 걸렸다. 변환 사전 구축 이후에는 더 빠르게 연산이 가능해질 예정

In [ ]:
len(menu_cluster)

51437

### 단순 초성 비율 기준 표준화 결과

- 결과는 엉망이다.

- 2글자가 대다수인 토큰이라서 겹치는게 많다

In [ ]:
menu_cluster

{'정식': '정식',
 '짜장면': '치즈 면',
 '짬뽕': '짬뽕',
 '우동': '모듬',
 '울면': '양념',
 '간짜장': '치즈',
 '사천짜장': '세트 치즈',
 '삼선짜장': '세트 치즈',
 '유니짜장': '양념 치즈',
 '삼선우동': '세트 모듬',
 '삼선짬뽕': '세트 짬뽕',
 '볶음밥': '볶음',
 '잡채밥': '초밥',
 '짜장밥': '전복',
 '짬뽕밥': '짬뽕',
 '오므라이스': '아메리카노',
 '삼선볶음밥': '세트 볶음',
 '잡탕밥': '전복',
 '군만두': '모듬',
 '쟁반짜장': '갈비찜',
 '사천쟁반짜장': '세트 갈비찜',
 '탕수육': '새우',
 '소고기탕수육': '불고기 새우',
 '라조기': '전골',
 '라조육': '오징어',
 '깐풍기': '깐풍',
 '깐풍육': '커피',
 '난자완스': '오징어',
 '팔보채': '피치',
 '양장피': '오징어',
 '짬뽕국': '짬뽕',
 '잡채': '정식',
 '고추잡채': '김치찌개',
 '계란탕': '구이 탕',
 '유산슬': '순살',
 '새우튀김': '세트 추가',
 '깐풍새우': '파스타',
 '치즈타르트': '추가 토마토',
 '초코케잌': '추가 새우',
 '찹쌀파이': '추가 새우',
 '크림바게트': '크림 불고기',
 '통팥빵': '튀김 빵',
 '산딸기파이': '소고기 새우',
 '깔끔추어탕': '꼬치 매운탕',
 '걸죽추어탕': '치즈 매운탕',
 '전주비빔밥': '치즈 볶음밥',
 '모듬회': '모듬 한',
 '잡어': '새우',
 '매운탕': '매운',
 '야채김밥': '양념 구이',
 '참치마요김밥': '추가 새우 구이',
 '새우날치알김밥': '세트 녹차 구이',
 '참치볶음밥': '추가 볶음',
 '야채비빔밥': '양념 볶음밥',
 '된장찌개': '김치찌개',
 '순두부찌개': '된장찌개',
 '돌솥비빔밥': '샤브샤브',
 '육개장': '간장',
 '등심돈까스': '정식 돈까스',
 '치즈돈까스': '추가 돈까스'

### 초/중/종성까지 전부 고려한 표준화

In [ ]:
menu_cluster

{'정식': '정식',
 '짜장면': '짜장 면',
 '짬뽕': '짬뽕',
 '우동': '우동',
 '울면': '라면',
 '간짜장': '짜장',
 '사천짜장': '사천 짜장',
 '삼선짜장': '삼선 짜장',
 '유니짜장': '짜장',
 '삼선우동': '삼선 우동',
 '삼선짬뽕': '삼선 짬뽕',
 '볶음밥': '볶음밥',
 '잡채밥': '잡채 밥',
 '짜장밥': '짜장밥',
 '짬뽕밥': '짬뽕 밥',
 '오므라이스': '아이스',
 '삼선볶음밥': '삼선 볶음밥',
 '잡탕밥': '잡탕 밥',
 '군만두': '만두',
 '쟁반짜장': '쟁반 짜장',
 '사천쟁반짜장': '사천 쟁반 짜장',
 '탕수육': '탕수육',
 '소고기탕수육': '소고기 탕수육',
 '라조기': '고기',
 '라조육': '라조육',
 '깐풍기': '깐풍',
 '깐풍육': '깐풍',
 '난자완스': '난자완스',
 '팔보채': '갈비',
 '양장피': '양파',
 '짬뽕국': '짬뽕 국',
 '잡채': '잡채',
 '고추잡채': '고추 잡채',
 '계란탕': '계란 탕',
 '유산슬': '유산슬',
 '새우튀김': '새우 튀김',
 '깐풍새우': '깐풍 새우',
 '치즈타르트': '치즈',
 '초코케잌': '초코 케이크',
 '찹쌀파이': '찹쌀 파이',
 '크림바게트': '크림 바게트',
 '통팥빵': '통팥 탕',
 '산딸기파이': '산딸기 파이',
 '깔끔추어탕': '추어탕',
 '걸죽추어탕': '추어탕',
 '전주비빔밥': '비빔밥',
 '모듬회': '모듬 회',
 '잡어': '장어',
 '매운탕': '매운탕',
 '야채김밥': '야채 김밥',
 '참치마요김밥': '참치 김밥',
 '새우날치알김밥': '새우 날치알 김밥',
 '참치볶음밥': '참치 볶음밥',
 '야채비빔밥': '야채 비빔밥',
 '된장찌개': '된장찌개',
 '순두부찌개': '두부찌개',
 '돌솥비빔밥': '돌솥 비빔밥',
 '육개장': '육개장',
 '등심돈까스': '등심 돈까스',
 '치즈돈까스': '

TO-DO

- 표준화 적용 대상 줄이기

-> 전체 적용시 비슷한 단어가 섞임

- OOV_list 중복값 제거, 딕셔너리화 고려

- 일련의 파이프라인에서 향후 EDA를 적용할 미분류 값들에 대한 정의

In [ ]:
len(OOV_list)

33252

In [ ]:
# 분류된 메뉴를 DataFrame화 
df_menu = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
df_menu['category'].unique()

array(['정식', '짜장 면', '짬뽕', ..., '삼선 해물 누룽지 탕', '파인 새우', '좌종당계'],
      dtype=object)

In [ ]:
len(df_menu['category'].unique())

37449

In [ ]:
pd.set_option('display.max_row', 100)

In [ ]:
df_menu.sample(100)

,title,category
21544,냉열무국수,열무 국수
41946,만조크림리조또,크림 리조또
17785,가래떡구이,가래떡 구이
29063,매운불닭,매운 불닭
44421,우렁된장찌게,우렁 된장 찌게
47184,매운해령고볶음,매운 볶음
41740,크리스피치킨샐러드,크리스피 치킨 샐러드
29395,돼지김치두부,돼지 김치 두부
37033,푸줏간돼지한판,푸줏간 돼지
32683,콜라변경,콜라 변경


### 생각할 부분

- 예외처리 부분 + OOV로 분류하기 전에 표준화

- 

- 